##### IMPORTAR AS PLANILHAS

In [ ]:
import numpy as np
import pandas as pd
from resolve_path import ajuste_path, read_input

# # Para pegar os dados da API da Open Meteo (comentado para nao pesar a pipeline)
# import time
# import openmeteo_requests
# import requests_cache
# from openmeteo_requests import Client
# from retry_requests import retry
# import requests

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "data/util/os/"

path_iw = ajuste_path(path_iw)

df_iw47 = pd.read_csv(path_iw + "IW47_Executadas_preparado.csv")

df_iw47.columns

In [ ]:
df_iw47

In [ ]:
colunas_tipo_atividade = ["Txt.breve operação", "Denominação TAM"]

In [ ]:
df_iw47["Data fim"] = pd.to_datetime(df_iw47["Data fim"])
df_iw47["Data inicio"] = pd.to_datetime(df_iw47["Data inicio"])
df_iw47["Local de instalação"] = df_iw47["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw47.loc[df_iw47["Data fim"]
                                        >= df_iw47["Data inicio"]]

num_linhas_removidas = df_iw47.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS
df = df_sem_linhas_irregulares

df["Meses"] = (
    (df["Data fim"].dt.year - df["Data inicio"].dt.year) * 12
    + (df["Data fim"].dt.month - df["Data inicio"].dt.month)
) + 1
df["HH por mes"] = df["HH final"] / df["Meses"]

##### ADICIONANDO DURAÇÃO DA OS ANTES DE AGRUPAR PARA MAIOR ACURÁCIA

In [ ]:
# Supondo que as colunas 'Data inicio' e 'Data fim' já estão no formato datetime64
df['Data inicio'] = pd.to_datetime(df['Data inicio'])
df['Data fim'] = pd.to_datetime(df['Data fim'])

# Criar uma condição para verificar se a Data fim é 2020 ou mais recente e Data inicio é 2019 ou anterior
condicao = (df['Data fim'] >= pd.Timestamp('2020-01-01')
            ) & (df['Data inicio'] < pd.Timestamp('2020-01-01'))

# Aplicar a condição para ajustar a Data inicio
df.loc[condicao, 'Data inicio'] = pd.Timestamp('2020-01-01')

# Calcular a duração em dias úteis e multiplicar por 8 para horas
df['Duração'] = (np.busday_count(df['Data inicio'].values.astype('datetime64[D]'),
                                 # 8 horas por dia
                                 df['Data fim'].values.astype('datetime64[D]')) + 1)*8

# Calcular a quantidade de meses após 2020 visto que mudamos a data de inicio
df['Meses após 2020'] = (df['Data fim'].dt.year - df['Data inicio'].dt.year) * \
    12 + (df['Data fim'].dt.month - df['Data inicio'].dt.month + 1)

# Calcular a duração em meses
df['Duração'] = df['Duração'] / df['Meses após 2020']

##### FUNCAO QUE AGRUPA HH POR ANO, MES, LOCAL E TIPO DE ATIVIDADE

In [ ]:
df["Data inicio"] = df["Data inicio"].apply(lambda x: x.replace(day=1))
df["Data fim"] = df["Data fim"].apply(lambda x: x.replace(day=1))

In [ ]:
# Gerar o espaço de tempo
espaco_de_tempo = pd.date_range(
    start='2020-01-01', end=pd.Timestamp.now(), freq="MS")
espaco_de_tempo = pd.DataFrame(
    {'merge': [1]*len(espaco_de_tempo), 'Ano Mes': espaco_de_tempo})

# Marcar operações que duram mais de um mês
df.loc[df['Meses'] > 1, 'dura mais de mes'] = 1

# Manter apenas as colunas necessárias
colunas_a_manter = [
    'Local de instalação',
    'Latitude',
    'Longitude',
    'HH por mes',
    'Meses',
    'dura mais de mes',
    'Data inicio',
    'Data fim',
    'Duração'
]
colunas_a_manter.extend(colunas_tipo_atividade)
df = df[colunas_a_manter]

# Expansão das operações que duram mais de um mês
df_expandido = pd.merge(df, espaco_de_tempo, left_on='dura mais de mes',
                        right_on='merge', how='left').drop(columns=['merge'])

# Tratamento das colunas de data para garantir o correto agrupamento
df_expandido['dura mais de mes'] = df_expandido['dura mais de mes'].fillna(0)
df_expandido.loc[df_expandido['dura mais de mes']
                 == 0, 'Ano Mes'] = df_expandido['Data fim']
df_expandido['Ano Mes'] = df_expandido['Ano Mes'].dt.to_period('M')
df_expandido['Data inicio'] = df_expandido['Data inicio'].dt.to_period('M')
df_expandido['Data fim'] = df_expandido['Data fim'].dt.to_period('M')

# Eliminar linhas onde a operação não estava ativa
df_expandido = df_expandido[(df_expandido['Ano Mes'] >= df_expandido['Data inicio']) &
                            (df_expandido['Ano Mes'] <= df_expandido['Data fim'])]

# Manter apenas as colunas necessárias para o agrupamento final
colunas_a_manter = ['Ano Mes', 'Local de instalação',
                    'Latitude', 'Longitude', 'HH por mes', 'Duração']
colunas_a_manter.extend(colunas_tipo_atividade)
df_expandido = df_expandido[colunas_a_manter]

# Para cada categoria de atividade, criar uma nova coluna para armazenar o HH dessa categoria
for coluna_tipo_atividade in colunas_tipo_atividade:
    for atividade in df_expandido[coluna_tipo_atividade].unique():
        if "HH de " + atividade in df_expandido.columns:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "HH de " + atividade + " de " + coluna_tipo_atividade] = df_expandido["HH por mes"]
        else:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "HH de " + atividade] = df_expandido["HH por mes"]

# Limpa colunas desnecessárias após o agrupamento
df_agrupado = df_expandido.groupby(
    ['Ano Mes', 'Local de instalação']).sum().reset_index()
df_agrupado.drop(columns=colunas_tipo_atividade, inplace=True)
df_agrupado.rename(columns={'HH por mes': 'HH total'}, inplace=True)

# Filtra para considerar apenas registros a partir de 2020-01
df_agrupado = df_agrupado[df_agrupado['Ano Mes'] >= '2020-01']


for li in df_agrupado["Local de instalação"].unique():
    df_agrupado.loc[df_agrupado['Local de instalação'] == li,
                    'Latitude'] = df_expandido.loc[df_expandido['Local de instalação'] == li, 'Latitude'].values[0]
    df_agrupado.loc[df_agrupado['Local de instalação'] == li,
                    'Longitude'] = df_expandido.loc[df_expandido['Local de instalação'] == li, 'Longitude'].values[0]

##### TRATAMENTO FINAL PARA PODER EXPORTAR A TABELA

In [ ]:
df_agrupado = df_agrupado.sort_values(
    by=["Ano Mes", "Local de instalação"]).reset_index(drop=True)

df_agrupado.fillna(0, inplace=True)

In [ ]:
df_agrupado

### Adicionando Colunas Metereologicas no Dataset de OS:

In [ ]:
# # Função de retry personalizada
# def retry(session, retries=5, backoff_factor=0.2):
#     def wrapped_request(method, url, **kwargs):
#         for attempt in range(retries):
#             try:
#                 response = session.request(method, url, **kwargs)
#                 response.raise_for_status()  # Levanta um erro se o status não for 200
#                 return response
#             except requests.exceptions.RequestException as e:
#                 if attempt == retries - 1:
#                     raise
#                 time.sleep(backoff_factor * (2 ** attempt))
#     return wrapped_request

In [ ]:
# import pandas as pd
# import requests_cache
# from openmeteo_requests import Client

# # Configurar a sessão da API com cache
# cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
# openmeteo = Client(session=cache_session)

# # Preparar um DataFrame para armazenar os resultados
# resultados = []

# # Iterar sobre cada linha do df_agrupado
# for index, row in df_agrupado.iterrows():
#     # Converter a coluna 'Ano Mes' para string
#     ano_mes = str(row['Ano Mes'])

#     # Dividir a string em ano e mês usando o espaço
#     ano, mes = ano_mes.split("-")
#     latitude = row['Latitude']
#     longitude = row['Longitude']

#     # Definir a data de início e fim do mês
#     start_date = f"{ano}-{mes}-01"
#     end_date = f"{ano}-{mes}-{pd.Period(start_date).days_in_month}"

#     # Parâmetros para a solicitação da API
#     params = {
#         "latitude": latitude,
#         "longitude": longitude,
#         "start_date": start_date,
#         "end_date": end_date,
#         "daily": [
#             "weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
#             "precipitation_sum", "rain_sum", "precipitation_hours",
#             "wind_speed_10m_max", "wind_gusts_10m_max"
#         ],
#         "timezone": "America/Sao_Paulo"
#     }

#     # Solicitar dados à API
#     try:
#         response = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)[0]

#         # Processamento dos dados diários
#         daily = response.Daily()
#         daily_data = {
#             "date": pd.to_datetime(daily.Time(), unit="s", utc=True),
#             "weather_code": daily.Variables(0).ValuesAsNumpy(),
#             "temperature_2m_max": daily.Variables(1).ValuesAsNumpy(),
#             "temperature_2m_min": daily.Variables(2).ValuesAsNumpy(),
#             "temperature_2m_mean": daily.Variables(3).ValuesAsNumpy(),
#             "precipitation_sum": daily.Variables(4).ValuesAsNumpy(),
#             "rain_sum": daily.Variables(5).ValuesAsNumpy(),
#             "precipitation_hours": daily.Variables(6).ValuesAsNumpy(),
#             "wind_speed_10m_max": daily.Variables(7).ValuesAsNumpy(),
#             "wind_gusts_10m_max": daily.Variables(8).ValuesAsNumpy(),
#         }

#         daily_dataframe = pd.DataFrame(data=daily_data)

#         # Calcular as médias das variáveis meteorológicas para o mês
#         monthly_averages = daily_dataframe.mean()

#         # Adicionar os resultados ao DataFrame final
#         resultados.append({
#             "Ano-Mes": ano_mes,
#             "Local de instalação": row['Local de instalação'],
#             "Latitude": latitude,
#             "Longitude": longitude,
#             "HH total": row['HH total'],
#             "Temp_Max": monthly_averages['temperature_2m_max'],
#             "Temp_Min": monthly_averages['temperature_2m_min'],
#             "Temp_Mean": monthly_averages['temperature_2m_mean'],
#             "Precipitation_Sum": monthly_averages['precipitation_sum'],
#             "Rain_Sum": monthly_averages['rain_sum'],
#             "Precipitation_Hours": monthly_averages['precipitation_hours'],
#             "Wind_Speed_Max": monthly_averages['wind_speed_10m_max'],
#             "Wind_Gusts_Max": monthly_averages['wind_gusts_10m_max']
#         })

#     except Exception as e:
#         print(f"Erro ao solicitar dados da API para {row['Local de instalação']} em {ano_mes}: {e}")

# # Converter a lista de resultados para um DataFrame
# df_resultados = pd.DataFrame(resultados)

# # Exportar para um arquivo CSV (opcional)
# # df_resultados.to_csv("resultados_meteorologicos.csv", index=False)

Fazer uma funcao q faz a api continuar de onde parou

In [ ]:
# df_resultados.to_csv(path_iw + "resultados_metereologicos_incompletos.csv", index=False)

In [ ]:
# # Carregar os resultados incompletos e o dataframe de treinamento
# df_resultados = pd.read_csv(path_iw + "resultados_metereologicos_incompletos.csv")
# df_agrupado = pd.read_csv(path_iw + "os_treinamento.csv")

# # Criar uma coluna de chave em ambos os DataFrames para comparação
# df_resultados['chave'] = df_resultados['Local de instalação'] + '-' + df_resultados['Ano-Mes']
# df_agrupado['chave'] = df_agrupado['Local de instalação'] + '-' + df_agrupado['Ano Mes'].astype(str)

# # Verificar quais registros já foram processados
# processados = df_resultados['chave'].tolist()

# # Filtrar o df_agrupado para pegar apenas as linhas que ainda não foram processadas
# faltantes = df_agrupado[~df_agrupado['chave'].isin(processados)]

# # Exibir os locais e datas que faltam processar
# print(f"Total de registros faltantes: {len(faltantes)}")
# print(faltantes[['Local de instalação', 'Ano Mes', 'Latitude', 'Longitude']])

# # Você pode agora usar o DataFrame `faltantes` para fazer as requisições restantes à API.

In [ ]:
# # Configurar a sessão da API com cache
# cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
# openmeteo = Client(session=cache_session)

# # Definir o tamanho do lote e o tempo de pausa entre os lotes (em segundos)
# lote_tamanho = 50  # Por exemplo, 50 registros por lote
# tempo_pausa = 5  # Pausa para respeitar o limite de API (60 segundos)

# # Preparar um DataFrame para armazenar os novos resultados
# novos_resultados = []

# # Iterar sobre os registros faltantes em lotes
# for i in range(0, len(faltantes), lote_tamanho):
#     lote_faltantes = faltantes.iloc[i:i + lote_tamanho]

#     # Fazer requisições para cada registro no lote
#     for index, row in lote_faltantes.iterrows():
#         ano_mes = str(row['Ano Mes'])
#         ano, mes = ano_mes.split("-")
#         latitude = row['Latitude']
#         longitude = row['Longitude']

#         # Definir a data de início e fim do mês
#         start_date = f"{ano}-{mes}-01"
#         end_date = f"{ano}-{mes}-{pd.Period(start_date).days_in_month}"

#         # Parâmetros para a solicitação da API
#         params = {
#             "latitude": latitude,
#             "longitude": longitude,
#             "start_date": start_date,
#             "end_date": end_date,
#             "daily": [
#                 "weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean",
#                 "precipitation_sum", "rain_sum", "precipitation_hours",
#                 "wind_speed_10m_max", "wind_gusts_10m_max"
#             ],
#             "timezone": "America/Sao_Paulo"
#         }

#         # Solicitar dados à API
#         try:
#             response = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)[0]

#             # Processar os dados diários
#             daily = response.Daily()
#             daily_data = {
#                 "date": pd.to_datetime(daily.Time(), unit="s", utc=True),
#                 "weather_code": daily.Variables(0).ValuesAsNumpy(),
#                 "temperature_2m_max": daily.Variables(1).ValuesAsNumpy(),
#                 "temperature_2m_min": daily.Variables(2).ValuesAsNumpy(),
#                 "temperature_2m_mean": daily.Variables(3).ValuesAsNumpy(),
#                 "precipitation_sum": daily.Variables(4).ValuesAsNumpy(),
#                 "rain_sum": daily.Variables(5).ValuesAsNumpy(),
#                 "precipitation_hours": daily.Variables(6).ValuesAsNumpy(),
#                 "wind_speed_10m_max": daily.Variables(7).ValuesAsNumpy(),
#                 "wind_gusts_10m_max": daily.Variables(8).ValuesAsNumpy(),
#             }

#             daily_dataframe = pd.DataFrame(data=daily_data)

#             # Calcular as médias das variáveis meteorológicas para o mês
#             monthly_averages = daily_dataframe.mean()

#             # Adicionar os resultados ao DataFrame final
#             novos_resultados.append({
#                 "Ano-Mes": ano_mes,
#                 "Local de instalação": row['Local de instalação'],
#                 "Latitude": latitude,
#                 "Longitude": longitude,
#                 "HH total": row['HH total'],
#                 "Temp_Max": monthly_averages['temperature_2m_max'],
#                 "Temp_Min": monthly_averages['temperature_2m_min'],
#                 "Temp_Mean": monthly_averages['temperature_2m_mean'],
#                 "Precipitation_Sum": monthly_averages['precipitation_sum'],
#                 "Rain_Sum": monthly_averages['rain_sum'],
#                 "Precipitation_Hours": monthly_averages['precipitation_hours'],
#                 "Wind_Speed_Max": monthly_averages['wind_speed_10m_max'],
#                 "Wind_Gusts_Max": monthly_averages['wind_gusts_10m_max']
#             })

#         except Exception as e:
#             print(f"Erro ao solicitar dados da API para {row['Local de instalação']} em {ano_mes}: {e}")

#     # Pausar entre os lotes para não exceder o limite de requisições
#     if (i + lote_tamanho) < len(faltantes):  # Apenas pausar se houver mais registros a serem processados
#         print(f"Pausando para evitar limite da API...")
#         time.sleep(tempo_pausa)
#         print(f"Voltando")

# # Converter a lista de resultados para um DataFrame
# df_novos_resultados = pd.DataFrame(novos_resultados)

# # Salvar os novos resultados em um CSV (opcional)
# df_novos_resultados.to_csv(path_iw + "novos_resultados_meteorologicos.csv", index=False)

In [ ]:
# # Combinar os dados novos com os resultados antigos
# df_todos_resultados = pd.concat([df_resultados, df_novos_resultados])

# # Salvar todos os resultados combinados (opcional)
# df_todos_resultados.to_csv(path_iw + "resultados_meteorologicos_completos.csv", index=False)

#### Adicionando Dados Meteorologicos Ja Coletados pela API da OpenMeteo

In [ ]:
# le resultados_meteorologicos_completos

path_ipt = "data/input/"

path_ipt = ajuste_path(path_ipt)

df_resultados_completos = pd.read_csv(
    path_ipt + "resultados_meteorologicos_completos.csv")

df_resultados_completos

In [ ]:
# Supondo que df_resultados_completos já esteja carregado
df_resultados_completos = df_resultados_completos.drop(columns=['chave'])

# Renomear a coluna 'Ano-Mes' para 'Ano Mes'
df_resultados_completos = df_resultados_completos.rename(
    columns={'Ano-Mes': 'Ano Mes'})

# Verifique o DataFrame para garantir que a coluna foi renomeada
print(df_resultados_completos.head())

##### EXPORTANDO PLANILHA

In [ ]:
# Verificar o tipo de dados da coluna 'Ano Mes' em df_resultados_completos
print(df_resultados_completos['Ano Mes'].dtype)

# Verificar o tipo de dados da coluna 'Ano Mes' em df_agrupado
print(df_agrupado['Ano Mes'].dtype)

In [ ]:
# Converter a coluna 'Ano Mes' para string em ambos os DataFrames
df_resultados_completos['Ano Mes'] = df_resultados_completos['Ano Mes'].astype(
    str)
df_agrupado['Ano Mes'] = df_agrupado['Ano Mes'].astype(str)

In [ ]:
# Supondo que df_resultados_completos e df_agrupado já estejam carregados

# Realizar a junção (merge) com base na coluna-chave
df_merged = df_resultados_completos.merge(df_agrupado, on=[
                                          'Ano Mes', 'Local de instalação', 'Latitude', 'Longitude'], how='right')

# Verifique o resultado da concatenação
print(df_merged.head())

In [ ]:
# Dropar a coluna 'HH total_x'
df_merged = df_merged.drop(columns=['HH total_x'])

# Renomear a coluna 'HH total_y' para 'HH total'
df_merged = df_merged.rename(columns={'HH total_y': 'HH total'})

# Verifique o resultado
print(df_merged.head())

In [ ]:
df_agrupado = df_merged

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado

In [ ]:
df_agrupado.to_csv(path_iw + "os_treinamento.csv", index=False)